## CB: tf-idf, autoencoder
새로운 유저, 새로운 콘텐츠에 대해서만 예측해야 하기 때문에 classic CF는 못쓴다. 특징 추출하고 mlp 예측 가자

시나리오
1. feature engineering & selection
2. mlp 학습
3. hyperparameter tuning
4. mlp 예측
5. 결과 제출

In [3]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import optuna

/opt/homebrew/Caskroom/miniconda/base/envs/recsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## load data

In [4]:
train_df = pd.read_csv("open/train.csv", index_col='ID')
test_df = pd.read_csv("open/test.csv", index_col='ID')
sample_submission_df = pd.read_csv("open/sample_submission.csv")

train_df.shape, test_df.shape, sample_submission_df.shape

((871393, 9), (159621, 8), (159621, 2))

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 871393 entries, TRAIN_000000 to TRAIN_871392
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User-ID              871393 non-null  object 
 1   Book-ID              871393 non-null  object 
 2   Book-Rating          871393 non-null  int64  
 3   Age                  871393 non-null  float64
 4   Location             871393 non-null  object 
 5   Book-Title           871393 non-null  object 
 6   Book-Author          871393 non-null  object 
 7   Year-Of-Publication  871393 non-null  float64
 8   Publisher            871393 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 66.5+ MB


## preprocess data

In [6]:
train_users = train_df['User-ID'].unique()
train_books = train_df['Book-ID'].unique()

len(train_users), len(train_books)

(83256, 243441)

In [7]:
test_users = test_df['User-ID'].unique()
test_books = test_df['Book-ID'].unique()

len(test_users), len(test_books)

(21909, 62333)

In [8]:
len(set(train_users) | set(test_users)), len(set(train_books) | set(test_books))

(92101, 270056)

새로운 유저, 새로운 책이 분명 있음

In [9]:
train_df.head()

,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
ID,,,,,,,,,
TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,2001.0,Mira
TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books
TRAIN_000002,USER_00000,BOOK_086781,0,23.0,"sackville, new brunswick, canada",Waverley (Penguin English Library),Walter Scott,1981.0,Penguin Books
TRAIN_000003,USER_00000,BOOK_098622,0,23.0,"sackville, new brunswick, canada",Mother Earth Father Sky,Sue Harrison,1991.0,Avon
TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,1989.0,Signet Book


User feature
 * Age: int
 * Location: string

Book feature
 * Book-Title: string
 * Book-Author: string
 * Year-Of-Pulication: int
 * Publisher: string

특징의 분해 이전에 label 형태로도 해볼 수 있음

# Feature Selection

In [10]:
features = test_df.columns
features

Index(['User-ID', 'Book-ID', 'Age', 'Location', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher'],
      dtype='object')

In [11]:
continuous_features = ['Age', 'Year-Of-Publication']
categorical_features = ['Location', 'Book-Title', 'Book-Author', 'Publisher']

In [12]:
train_df['Location'] = train_df['Location'].str.replace(' ', '')
train_df['Location_country'] = train_df['Location'].apply(lambda x: x.split(',')[-1])
train_df['Location_city'] = train_df['Location'].apply(lambda x: x.split(',')[-2])
train_df.drop(columns='Location', inplace=True)
train_df.head()

,User-ID,Book-ID,Book-Rating,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location_country,Location_city
ID,,,,,,,,,,
TRAIN_000000,USER_00000,BOOK_044368,8,23.0,Road Taken,Rona Jaffe,2001.0,Mira,canada,newbrunswick
TRAIN_000001,USER_00000,BOOK_081205,8,23.0,Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books,canada,newbrunswick
TRAIN_000002,USER_00000,BOOK_086781,0,23.0,Waverley (Penguin English Library),Walter Scott,1981.0,Penguin Books,canada,newbrunswick
TRAIN_000003,USER_00000,BOOK_098622,0,23.0,Mother Earth Father Sky,Sue Harrison,1991.0,Avon,canada,newbrunswick
TRAIN_000004,USER_00000,BOOK_180810,8,23.0,She Who Remembers,Linda Lay Shuler,1989.0,Signet Book,canada,newbrunswick


In [13]:
test_df['Location'] = test_df['Location'].str.replace(' ', '')
test_df['Location_country'] = test_df['Location'].apply(lambda x: x.split(',')[-1])
test_df['Location_city'] = test_df['Location'].apply(lambda x: x.split(',')[-2])
test_df.drop(columns='Location', inplace=True)
test_df.head()

,User-ID,Book-ID,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location_country,Location_city
ID,,,,,,,,,
TEST_000000,USER_00008,BOOK_047966,37.0,Birds of Prey: A Novel of Suspense,J.A. Jance,2002.0,Avon,usa,ohio
TEST_000001,USER_00008,BOOK_119494,37.0,Midnight Voices,JOHN SAUL,2003.0,Ballantine Books,usa,ohio
TEST_000002,USER_00008,BOOK_151775,37.0,Breaking Free : A Prescription for Personal an...,David M. Noer,1996.0,Jossey-Bass,usa,ohio
TEST_000003,USER_00008,BOOK_176255,37.0,Bitter Harvest,Ann Rule,1999.0,Pocket,usa,ohio
TEST_000004,USER_00008,BOOK_187307,37.0,Embraced by the Light,Betty J. Eadie,1994.0,Bantam Books,usa,ohio


# encoding

for location_country, publisher

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
cont_encoder = OneHotEncoder(handle_unknown='ignore')
train_country_array = cont_encoder.fit_transform(train_df[['Location_country']]).toarray()
test_country_array = cont_encoder.transform(test_df[['Location_country']]).toarray()

In [16]:
# pub_encoder = OneHotEncoder(handle_unknown='ignore')
# train_publisher_array = pub_encoder.fit_transform(train_df[['Publisher']]).toarray()
# test_publisher_array = pub_encoder.transform(train_df[['Publisher']]).toarray()

# scaling

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
scaler = StandardScaler()
train_cont_array = scaler.fit_transform(train_df[continuous_features])
test_cont_array = scaler.transform(test_df[continuous_features])

# merge prcessed data

In [19]:
train_cont_array.shape, train_country_array.shape#, train_publisher_array.shape

((871393, 2), (871393, 243))

In [20]:
train_x = np.concatenate([train_cont_array, train_country_array], axis=1) #train_publisher_array
test_x = np.concatenate([test_cont_array, test_country_array], axis=1) #test_publisher_array

train_x.shape, test_x.shape

((871393, 245), (159621, 245))

In [21]:
train_y = train_df['Book-Rating'].values
train_y.shape

(871393,)

In [22]:
# train_array = np.concatenate([train_cont_array, train_country_array, train_publisher_array], axis=1)
# test_array = np.concatenate([test_cont_array, test_country_array, test_publisher_array], axis=1)

## first Trial

In [23]:
mname = 'mlpregressor'

In [40]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer

In [41]:
params = {
    'hidden_layer_sizes': [100, 200], #10, 50, 
    'activation' : ['logistic', 'tanh', 'relu'], #'identity', 
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.001, 0.0001],#[1, 0.1, 0.01, 0.001, 0.0001],
    'batch_size': [10, 50, 100],#, 200, 500, 1000],
    'warm_start': [True, False],
    'learning_rate_init': [0.001, 0.0001], #24 added #1, 0.1, 0.01, 
    'max_iter': [1000, 5000], #24 added #100, 200, 500, 
    
}

In [42]:
def objective(trial):
    hidden_layer_sizes = trial.suggest_int("hidden_layer_sizes", 100, 200, log=True)
    activation = trial.suggest_categorical('activation', ['logistic', 'tanh', 'relu'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'sgd', 'adam'])
    alpha = trial.suggest_float("alpha", 0.0001, 0.001)
    batch_size = trial.suggest_int("rf_max_depth", 10, 100)
    warm_start = trial.suggest_categorical('warm_start', [True, False])
    learning_rate_init = trial.suggest_float("learning_rate_init", 0.0001, 0.001)
    max_iter = trial.suggest_int("max_iter", 1000, 5000)
    
    classifier_obj = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes, activation=activation,
        solver=solver, alpha=alpha, batch_size=batch_size,
        warm_start=warm_start, max_iter=max_iter,
        learning_rate_init=learning_rate_init,
    )
    score = make_scorer(mean_squared_error)
    score = cross_val_score(classifier_obj, train_x, train_y, n_jobs=-1, cv=5, scoring=score)
    
    return score

In [43]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2023-04-25 00:35:05,261] A new study created in memory with name: no-name-faef3ce7-93cd-45d2-95da-1b1cc79a542f
[W 2023-04-25 05:16:26,850] Trial 0 failed with parameters: {'hidden_layer_sizes': 118, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.0006079684675159964, 'rf_max_depth': 62, 'warm_start': True, 'learning_rate_init': 0.00047300361101877887, 'max_iter': 4392} because of the following error: The value array([15.08657094, 14.80368775, 14.80366675, 15.0588726 , 14.61848062]) could not be cast to float..
[W 2023-04-25 05:16:26,887] Trial 0 failed with value array([15.08657094, 14.80368775, 14.80366675, 15.0588726 , 14.61848062]).
[W 2023-04-25 11:38:45,730] Trial 1 failed with parameters: {'hidden_layer_sizes': 163, 'activation': 'tanh', 'solver': 'adam', 'alpha': 0.0009896394396562108, 'rf_max_depth': 88, 'warm_start': False, 'learning_rate_init': 0.00048270289627012447, 'max_iter': 1415} because of the following error: The value array([15.14500389, 14.76668462, 14.8296408

KeyboardInterrupt: 